In [1]:
import os
import urllib
import urllib.request

scv_blob_http_uri = "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output.ndjson.gz"

scv_blob_http_uris = [
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-0.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-1.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-2.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-3.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-4.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-5.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-6.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-7.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-8.ndjson.gz",
    "https://storage.googleapis.com/clingen-public/clinvar-gk-pilot/2024-04-07/dev/combined-scv_output-split/part-9.ndjson.gz",
]
scv_blob_http_uri = scv_blob_http_uris[0]


def download_file(url, local_filename, chunk_size=8*1024*1024):
    # Open the URL
    with urllib.request.urlopen(url) as response:
        with open(local_filename, "wb") as out_file:
            while True:
                chunk = response.read(chunk_size)
                if not chunk:
                    break
                out_file.write(chunk)


def content_length(url):
    with urllib.request.urlopen(url) as response:
        return int(response.getheader("Content-Length"))


scv_local_file_path = os.path.basename(scv_blob_http_uri)
if os.path.exists(scv_local_file_path) and os.path.getsize(scv_local_file_path) == content_length(scv_blob_http_uri):
    print(f"File {scv_local_file_path} already exists "
          "and is the correct size. Skipping download.")
    scv_file = scv_local_file_path
else:
    print(f"Downloading file from {
          scv_blob_http_uri} to {scv_local_file_path}")
    download_file(scv_blob_http_uri, scv_local_file_path)

scv_file = scv_local_file_path

In [2]:
import contextlib
import shutil
import gzip


def split(input_filename, output_directory, partitions):
    filenames = [f"part-{i}.ndjson.gz" for i in range(partitions)]
    file_paths = [
        os.path.join(output_directory, filename) for filename in filenames
    ]
    print(f"Splitting {input_filename} into {
          partitions} partitions: {file_paths}")
    with gzip.open(input_filename, "rt", encoding="utf-8") as f:
        filenames = [f"part-{i}.ndjson.gz" for i in range(partitions)]
        file_paths = [
            os.path.join(output_directory, filename) for filename in filenames
        ]
        with contextlib.ExitStack() as stack:
            files = [
                stack.enter_context(gzip.open(file_path, "wt"))
                for file_path in file_paths
            ]
            for i, line in enumerate(f):
                file_idx = i % partitions
                files[file_idx].write(line)
    return file_paths


# partition_count = 10
# split_dir = scv_local_file_path.replace(".ndjson.gz", "-split")
# if os.path.isdir(split_dir) and len(os.listdir(split_dir)) == partition_count:
#     print(f"Directory {split_dir} already exists. Skipping split.")
#     partitioned_paths = [os.path.join(split_dir, p)
#                          for p in os.listdir(split_dir)]
# else:
#     if os.path.isdir(split_dir):
#         shutil.rmtree(split_dir)
#     os.makedirs(split_dir, exist_ok=True)
#     partitioned_paths = split(scv_file, split_dir, partition_count)

# # Update the scv_file variable to just be the first partition.
# # The second pair of variants defined above are from that partition.
# scv_file = partitioned_paths[0]

In [3]:
def print_scvs(scvs):
    """
    Reused function to print some core fields from a GKS-modeled ClinVar SCV record.
    """
    for scv in scvs:
        classification = scv["classification"]["label"]
        condition = scv["condition"]["label"]
        print(f"SCV: {scv['id']} ")
        print(f"  Classification: {classification}")
        print(f"  Condition: {condition}")
        print()

Our ClinVar datasets are available as both id-keyed JSON files, and NDJSON files. For each format there is a variation file and an SCV file. The demos in this notebook use the NDJSON formatted files. The records of the variation file are `CategoricalVariation` objects, and the records of the SCV file are `VariationPathogenicity` (sub-class of `Statement`)

In [4]:
from nis import cat
import os
import gzip
import json

################################
# Query the SCV file for a VRS ID using vanilla Python
#
# - for a given ClinVar Variation ID, find the corresponding GA4GH CatVar record in the CatVar
#   file and find the SCVs which reference that variant in the SCV file
#
#   (NOTE: the SCV file also contains the full CatVar definition in the "variation" field, but
#    this example illustrates how to query across both files, since the SCV file can be
#    relationally normalized to extract that redundant entity and refer to the variant
#    by the CatVar ID as a foreign key)
#
# - print the SCV interpretations for that variant
#
################################
################################
# Inputs

################################
# A CanonicalAllele
# For searching based on the GKS Categorical Variation (CatVrs) ID
catvar_id_canonicalallele = "clinvar:2769522"
# For searching based on the GA4GH VRS Variation ID
vrs_id_canonicalallele = "ga4gh:VA.hf_va4AnlG99NuOjtaXJzh_XvszWWOO9"

# This one is in the first file if partitioned into 10 files roundrobin
# And it has 2 SCVs in that partition that refer to it.
catvar_id_canonicalallele = "clinvar:841556"
vrs_id_canonicalallele = "ga4gh:VA.IIvKzExYL-5JMr8lpBZ2G91Bl9T8eBeA"

################################
# A CategoricalCnv
# For searching based on the GKS Categorical Variation (CatVrs) ID
catvar_id_categoricalcnv = "clinvar:599353"
# For searching based on the GA4GH VRS Variation ID
vrs_id_categoricalcnv = "ga4gh:CX.5iqyOA4L5njh5FpymTPcwQ8oHTilQFmo"  # GRCh38 member

catvar_id_categoricalcnv = "clinvar:599187"
vrs_id_categoricalcnv = "ga4gh:CX.keI6gf2rN-zSQzVTsxhXmW9k2wGA9BbF"  # GRCh37 member
################################
assert os.path.exists(scv_file)

/var/folders/34/2bz41_vj29v65k2g7r4jfh100000gp/T/ipykernel_55674/2550988733.py:1: DeprecationWarning: 'nis' is deprecated and slated for removal in Python 3.13
  from nis import cat


In [5]:
################################
# Query the SCV file for the matching VRS ID
################################


def query_scvs_by_vrs_id(vrs_id: str, scv_file_name: str):
    scvs = []
    # catvars = []
    with gzip.open(scv_file_name, "rt") as f:
        for line in f:
            record = json.loads(line)
            variation = record["variation"]
            processing_errors = [
                e
                for e in variation.get("extensions", [])
                if e["name"] == "vrs processing errors"
            ]
            if len(processing_errors) > 0:
                # print(f"Skipping SCV record with VRS processing errors: {line}")
                continue

            match variation["type"]:
                case "CategoricalCnv":
                    if "members" not in variation:
                        # Unsupported?
                        # e.g. "clinvar:1878325"
                        # "NC_000018.9:g.(48556994_48573289)_48573471dup"
                        # raise ValueError(f"CategoricalCnv missing members field: {line}")
                        continue
                    members = variation["members"]
                    member_vrs_ids = [m["id"] for m in members]
                    if vrs_id in member_vrs_ids:
                        scvs.append(record)

                case "CanonicalAllele":
                    if "definingContext" not in variation:
                        # Unsupported allele type?
                        # e.g. clinvar:215984
                        # "NM_000251.2(MSH2):c.212-?_366+?dup"
                        # raise ValueError(f"CanonicalAllele missing definingContext field: {line}")
                        continue
                    if variation["definingContext"]["id"] == vrs_id:
                        scvs.append(record)
                case "DescribedVariation":
                    # not an error in processing, but does not have any VRS IDs
                    continue
                    # raise ValueError(f"DescribedVariation not yet implemented: {line}")
                case _:
                    raise ValueError(
                        f"Unexpected variation type ({variation['type']}): {
                            line}"
                    )
    return scvs

In [6]:
################################
# Query the SCV file for the matching CatVar ID
################################


def query_scvs_by_catvar_id(catvar_id: str, scv_file_name: str):
    scvs = []
    # catvars = []
    with gzip.open(scv_file_name, "rt") as f:
        for line in f:
            record = json.loads(line)
            variation = record["variation"]
            record_catvar_id = variation["id"]

            if record_catvar_id == catvar_id:
                scvs.append(record)

    return scvs

In [7]:
scvs_by_vrs_id_canonicalallele = query_scvs_by_vrs_id(
    vrs_id_canonicalallele, scv_file)

print_scvs(scvs_by_vrs_id_canonicalallele)

SCV: SCV002082310.1 
  Classification: Pathogenic
  Condition: Autosomal recessive limb-girdle muscular dystrophy type 2B

SCV: SCV001207574.5 
  Classification: Pathogenic
  Condition: Qualitative or quantitative defects of dysferlin



In [8]:
scvs_by_vrs_id_categoricalcnv = query_scvs_by_vrs_id(
    vrs_id_categoricalcnv, scv_file)

print_scvs(scvs_by_vrs_id_categoricalcnv)

SCV: SCV000863548.1 
  Classification: Pathogenic
  Condition: 1q24q25 microdeletion syndrome



In [9]:
scvs_by_catvar_id_canonicalallele = query_scvs_by_catvar_id(
    catvar_id_canonicalallele, scv_file
)

print_scvs(scvs_by_catvar_id_canonicalallele)

SCV: SCV002082310.1 
  Classification: Pathogenic
  Condition: Autosomal recessive limb-girdle muscular dystrophy type 2B

SCV: SCV001207574.5 
  Classification: Pathogenic
  Condition: Qualitative or quantitative defects of dysferlin



In [10]:
scvs_by_catvar_id_categoricalcnv = query_scvs_by_catvar_id(
    catvar_id_categoricalcnv, scv_file
)

print_scvs(scvs_by_catvar_id_categoricalcnv)

SCV: SCV000863548.1 
  Classification: Pathogenic
  Condition: 1q24q25 microdeletion syndrome

